## Model Definition

In [26]:
import torch
import torch.nn as nn
from torchvision import models
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, backbone, num_classes=-1):
        super(Net, self).__init__()
        self.resnet_layer = nn.Sequential(*list(backbone.children())[:-1])
        self.num_classes = num_classes # when num_classes==-1, return the features instead of the logits
        if num_classes != -1:
            self.linear = nn.Linear(2048, num_classes)

    def forward(self, x):
        x = self.resnet_layer(x)
        if self.num_classes != -1:
            x = torch.flatten(x, 1)
            x = self.linear(x)

        return x

group_norm = lambda channels: nn.GroupNorm(32, channels)
resnet = models.resnet50(norm_layer=group_norm, pretrained=False)
pretrain_path = "./network-backbone_torchvision.pkl"
pretrained_dict = torch.load(pretrain_path)
resnet.load_state_dict(pretrained_dict, strict=False)
net = Net(resnet, 2)

## Dataset Definition

In [ ]:
import os
import random
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import Dataset

PIXEL_MEAN = [0.5, 0.5, 0.5]
PIXEL_STD = [0.5, 0.5, 0.5]
IMAGE_H = IMAGE_W = 256
class_name = ['NORM', 'TUM']

test_transform = transforms.Compose(
     [transforms.ToTensor(),
     transforms.Normalize(tuple(PIXEL_MEAN), tuple(PIXEL_STD))])

class CRCDataset(Dataset):
    def __init__(self, mode, img_path, data_transform, class_name):
        self.mode = mode
        self.transform = data_transform
        self.img_path = img_path
        image_name_file = os.path.join(img_path, f'{mode}.lst')
        with open(image_name_file, 'r') as f:
            subjects = f.readlines()
        self.list_image_path = [subject.strip() for subject in subjects if subject.split('/')[1] in class_name]
        print(f'len of dataset {mode}: {len(self.list_image_path)}')
        self.data_size = len(self.list_image_path)
        self.class_name = class_name

    def __getitem__(self, item):
        img_name = self.list_image_path[item]
        image = Image.open(os.path.join(self.img_path, img_name.replace('./', '')))
        image = image.resize((IMAGE_H, IMAGE_W)) # Crop might be better here
        label = self.label[item]
        return self.transform(image), torch.LongTensor([label])

    def __len__(self):
        return self.data_size

testset = CRCDataset('test', '../crc_data', test_transform, class_name)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=1)

## Get Features or Classification Results

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        outputs = net(images)